In [0]:
!pip install omdb
!pip install senticnet
!pip install tqdm
!pip install textacy
!pip install dframcy

In [0]:
import re
import json
import pandas as pd
import numpy as np
import spacy
import textacy
import textacy.preprocessing

from dframcy import DframCy
from tqdm import tqdm
from collections import Counter
from senticnet.senticnet import SenticNet

sn = SenticNet()

In [0]:
file = 'Save_3.txt'

with open(file, mode='r+', encoding="utf-8") as script:
	developer = json.load(script)
script.close()
developer = re.sub(r'null','None',developer)

DF1t = pd.DataFrame.from_dict(eval(developer))
DF_ScriptPersoFilm = DF1t.T

Speeches base

In [0]:

# extraire un dicto from df_srcipt -> key name, values speeches
#Speeches = DF_ScriptPersoFilm['Speech'].tolist()
Names = DF_ScriptPersoFilm['Character'].tolist()

valid_tag = ['ADJ', 'ADV', 'INTJ', 'NOUN', 'ADP']

Speeches alt

In [0]:
with open('/content/drive/My Drive/CleanSpeeches2.txt', mode="r", encoding="utf-8") as script:
  Speeches = eval(script.read())
script.close()

corpus

In [0]:
corpus = textacy.Corpus("en_core_web_sm", data=Speeches)

In [0]:
print(type(corpus))

<class 'textacy.corpus.Corpus'>


In [0]:
#pickle
import pickle

In [0]:
pickle.dump( corpus, open( "/content/drive/My Drive/corpus2.p", "wb" ) )

In [0]:
#corpus = pickle.load(open("/content/drive/My Drive/corpus.p", "r", encoding="utf-8"))
with open("/content/drive/My Drive/corpus.p", "rb") as input_file:
  corpus = pickle.load(input_file)

In [0]:
from textacy.vsm.vectorizers import Vectorizer

In [0]:
tokenized_docs = (doc._.to_terms_list(ngrams=[1,2], entities=False, as_strings=True, include_pos=valid_tag, normalize='lemma', filter_stops=True, filter_punct=True) for doc in corpus)
vectorizer = Vectorizer()
doc_term_matrix = vectorizer.fit_transform(tokenized_docs)

In [0]:
Sum = doc_term_matrix.sum(axis=1)

In [0]:
# utiliser numpy
DFTest = pd.DataFrame.sparse.from_spmatrix(doc_term_matrix)

In [0]:
DFTest.columns = vectorizer.terms_list

In [0]:
print(DFTest.head())

   -PRON-  -PRON- big  AWOL  ...  zucchini uh  zygotic  zygotic bone
0       0           0     0  ...            0        0             0
1       0           0     0  ...            0        0             0
2       0           0     0  ...            0        0             0
3       0           0     0  ...            0        0             0
4       0           0     0  ...            0        0             0

[5 rows x 175866 columns]


In [0]:
ToRequest = vectorizer.terms_list
#100% des concepts

In [0]:
Cnt_firstcatch_1n = 0
Cnt_firstcatch_2n = 0
Cnt_Secondcatch_1n = 0
Cnt_Secondtcatch_2n = 0
Cnt_firstcatch_2n_1n = 0
Cnt_NotCatch_Relative_1n = 0
Cnt_NotCatch_Relative_2n = 0
Cnt_NotCatch_vocab_1n = 0
Cnt_NotCatch_vocab_2n = 0
Total = len(ToRequest)

In [0]:
Cnt_1n = 0
Cnt_2n = 0
for elem in ToRequest:
  splited = elem.split()
  if len(splited) == 1:
    Cnt_1n += 1
  else:
    Cnt_2n += 1

#Gensim Model

In [0]:
!apt install enchant
!pip install enchant

In [0]:
import enchant
d = enchant.Dict('en_US')

In [0]:
# imports needed and logging
import gzip
import gensim 
import logging
import json

In [0]:
CleanSpeeches = []
count = 0
for speech in Speeches:
  if count % 500 == 0:
    print(count)
  CleanSp = gensim.utils.simple_preprocess(speech)
  try:
    if type(CleanSp[0]) is list:
      CleanSp = [item for sublist in CleanSp for item in sublist]
  except:
    print("A bugg with :")
    print(CleanSp)
    CleanSpeeches.append('')
    continue
  out = ''
  for word in CleanSp:
    if d.check(word) == True:
      out = out + word + " "
    else:
      remplacement = d.suggest(word)
      if len(remplacement) >= 1:
        out = out + remplacement[0] + " "
      else:
        print(word + "as been droped out")
  CleanSpeeches.append(out)
  count += 1

In [0]:
from senticnet5 import senticnet
key_list = []

for key in senticnet.keys():
  new_key = re.sub(r'_',' ', key)
  senticnet[new_key] = senticnet.pop(key)
  key_list.append(new_key)

In [0]:
Merged_listes = Speeches + key_list

data = [gensim.utils.simple_preprocess(d) for d in Merged_listes]

In [0]:
import gensim.models

model = gensim.models.Word2Vec(data, min_count=1)

In [0]:
for i, word in enumerate(model.wv.vocab):
    if i == 100:
        break
    print(word)

In [0]:
bob = "aardvark"
if bob in model.wv.vocab:
  print("oui")
else:
  print("non")

non


In [0]:
print(model.wv.most_similar(positive="oh"))

[('um', 0.7027236819267273), ('wow', 0.7005138993263245), ('darling', 0.697024405002594), ('huh', 0.6942256689071655), ('thanks', 0.6768074035644531), ('jesus', 0.6709182262420654), ('uh', 0.6629387140274048), ('dear', 0.660999596118927), ('damn', 0.6584579944610596), ('yeah', 0.6573576927185059)]


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


# out

In [0]:
#nb de concepts -> total, modifier , etc ajouter des compteurs 
catchElem = []
PowerLink = []
for elem in ToRequest:
	elem = elem.lower()
	splited = elem.split()
	if elem in senticnet.keys():
		sentics = sn.sentics(str(elem))
		catchElem.append(sentics)
		PowerLink.append(1)
		if len(splited) == 1:
			Cnt_firstcatch_1n += 1
		else:
			Cnt_firstcatch_2n += 1
		continue
	if len(splited) == 1:
		if elem in model.wv.vocab:
			relativeness = model.wv.most_similar(positive=elem)
			count = 1
			for relativ in relativeness:
				if relativ[0] in senticnet.keys():
					newtry = relativ[0]
					powerlink = relativ[1]
					sentics = sn.sentics(str(relativ[0]))
					catchElem.append(sentics)
					PowerLink.append(powerlink)
					Cnt_Secondcatch_1n += 1
					break
				else:
					if count >= len(relativeness):
						print(elem + "has no relatives in the senticsNet database")
						catchElem.append({'pleasantness': '0', 'attention': '0', 'sensitivity': '0', 'aptitude': '0'})
						PowerLink.append(0)
						Cnt_NotCatch_Relative_1n += 1
				count += 1
		else:
			print(elem + " is not in the w2v vocab")
			catchElem.append({'pleasantness': '0', 'attention': '0', 'sensitivity': '0', 'aptitude': '0'})
			PowerLink.append(0)
			Cnt_NotCatch_vocab_1n += 1
	else:
		_temp = []
		_tempPL = []
		for wrd in splited:
			if wrd in senticnet.keys():
				sentics_wrd = sn.sentics(str(wrd))
				_temp.append(sentics_wrd)
				_tempPL.append(1)
				Cnt_firstcatch_2n_1n += 1
				continue
			else:
				if wrd in model.wv.vocab:
					relativeness = model.wv.most_similar(positive=wrd)
					count = 1
					for relativ in relativeness:
						if relativ[0] in senticnet.keys():
							powerlink = relativ[1] 
							sentics_wrd = sn.sentics(str(relativ[0]))
							_temp.append(sentics_wrd)
							_tempPL.append(powerlink)
							Cnt_Secondtcatch_2n += 1
							break
						else:
							if count >= len(relativeness):
								print(wrd + "has no relatives in the senticsNet database")
								_temp.append({'pleasantness': '0', 'attention': '0', 'sensitivity': '0', 'aptitude': '0'})
								_tempPL.append(0)
								Cnt_NotCatch_Relative_2n += 1
						count += 1
				else:
					print(wrd + " is not in the w2v vocab")
					_temp.append({'pleasantness': '0', 'attention': '0', 'sensitivity': '0', 'aptitude': '0'})
					_tempPL.append(0)
					Cnt_NotCatch_vocab_2n += 1
		toadd = []
		catchElem.append({'pleasantness':	str((float(_temp[0]['pleasantness'])+		float(_temp[1]['pleasantness']))/2),\
		                  'attention':		str((float(_temp[0]['attention'])+			float(_temp[1]['attention']))/2),\
											'sensitivity':	str((float(_temp[0]['sensitivity'])+		float(_temp[1]['sensitivity']))/2),\
											'aptitude':			str((float(_temp[0]['aptitude'])+				float(_temp[1]['aptitude']))/2)})
		del toadd, _temp
		powerlink = 0
		for i in _tempPL:
			powerlink += i
		PowerLink.append(powerlink/2)
		del powerlink, _tempPL
	del splited
	#récupérer tous les outils possible pour l'argumentations futur

In [0]:
print("first catch 1N: " + str(Cnt_firstcatch_1n))
print("first catch 2N: " + str(Cnt_firstcatch_2n))
print("first catch 2N divided: " + str(Cnt_firstcatch_2n_1n/2))
print("1N: " + str(Cnt_1n))
print("2N: " + str(Cnt_2n))
print("total first catch: " + str((Cnt_firstcatch_1n + Cnt_firstcatch_2n)/Total))
print(Cnt_Secondcatch_1n)
print(Cnt_Secondtcatch_2n)
print(Cnt_NotCatch_Relative_1n)
print(Cnt_NotCatch_Relative_2n)
print(Cnt_NotCatch_vocab_1n)
print(Cnt_NotCatch_vocab_2n)
print(Total)
print(str(Total - Cnt_NotCatch_Relative_2n - Cnt_NotCatch_Relative_1n - Cnt_NotCatch_vocab_1n - Cnt_NotCatch_vocab_2n -Cnt_firstcatch_1n -Cnt_firstcatch_2n -Cnt_Secondcatch_1n -Cnt_Secondtcatch_2n))

first catch 1N: 15593
first catch 2N: 4300
first catch 2N divided: 131276.0
1N: 23399
2N: 152467
total first catch: 0.1131145303810856
7046
31764
244
1014
516
1004
175866
114385


In [0]:
Catch_info = {
    "First catch 1N" : Cnt_firstcatch_1n,
    "First catch 2N" : Cnt_firstcatch_2n,
    "First catch 2N_1n" : Cnt_firstcatch_2n_1n,
    "Total 1N" : Cnt_1n,
    "Total 2N" : Cnt_2n,
    "Second catch 1N" : Cnt_Secondcatch_1n,
    "Second catch 2N" : Cnt_Secondtcatch_2n,
    "Not catch rel 1N" : Cnt_NotCatch_Relative_1n,
    "Not catch rel 2N" : Cnt_NotCatch_Relative_2n,
    "Not catch voc 1N" : Cnt_NotCatch_vocab_1n,
    "Not catch voc 2N" : Cnt_NotCatch_vocab_2n,
    "Total" : Total
}

In [0]:
with open('/content/drive/My Drive/Catch_info2.txt', mode="r", encoding="utf-8") as script:
  Catch_info = eval(script.read())
script.close()

In [6]:
for key in Catch_info.keys():
  print(str(key) + ": " + str(Catch_info[key]) )

First catch 1N: 18355
First catch 2N: 8575
First catch 2N_1n: 513579
Total 1N: 29504
Total 2N: 299515
Second catch 1N: 9987
Second catch 2N: 64104
Not catch rel 1N: 550
Not catch rel 2N: 2373
Not catch voc 1N: 612
Not catch voc 2N: 1824
Total: 329019


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open('/content/drive/My Drive/Catch_info2.txt', 'w') as f:
  f.write(str(Catch_info))

In [0]:
with open('/content/drive/My Drive/catchElem2.txt', 'w') as f:
  f.write(str(catchElem))

In [0]:
with open('/content/drive/My Drive/PowerLink2.txt', 'w') as f:
  f.write(str(PowerLink))

In [0]:
print(DFTest.head())

   -PRON-  -PRON- big  AWOL  ...  zucchini uh  zygotic  zygotic bone
0       0           0     0  ...            0        0             0
1       0           0     0  ...            0        0             0
2       0           0     0  ...            0        0             0
3       0           0     0  ...            0        0             0
4       0           0     0  ...            0        0             0

[5 rows x 175866 columns]


In [0]:
DF_weight = pd.DataFrame.from_dict(catchElem)

In [0]:
DFT_weight = DF_weight.T

In [0]:
PowerLink = [ float(x) for x in PowerLink ]

In [0]:
DF_weight = DF_weight.apply(pd.to_numeric)

In [0]:
DF_PL_weight = DF_weight.T.mul(PowerLink)

In [0]:
DF_PL_weight = DF_PL_weight.T

In [0]:
import scipy

In [0]:
PL_weight = scipy.sparse.csr_matrix(DF_PL_weight.values)

In [0]:
print(type(doc_term_matrix))
print(type(Sum))
print(type(catchElem))
print(type(DF_PL_weight))
print(type(PL_weight))

<class 'scipy.sparse.csr.csr_matrix'>
<class 'numpy.matrix'>
<class 'list'>
<class 'pandas.core.frame.DataFrame'>
<class 'scipy.sparse.csr.csr_matrix'>


In [0]:
print(PL_weight.T.shape)
print(doc_term_matrix.shape)

(4, 175866)
(16327, 175866)


In [0]:
Mul_Matrix = doc_term_matrix.dot(PL_weight)

In [0]:
print(Mul_Matrix.shape)
print(Sum.shape)

(16327, 4)
(16327, 1)


In [0]:
Weight_Matrix = Mul_Matrix/Sum

/usr/local/lib/python3.6/dist-packages/scipy/sparse/base.py:599: RuntimeWarning: invalid value encountered in true_divide
  return np.true_divide(self.todense(), other)


In [0]:
Weight_Matrix_2 = np.true_divide(Mul_Matrix, Sum)

SystemError: ignored

In [0]:
print(Weight_Matrix[:5])

[[-0.057      -0.3985     -0.3         0.03      ]
 [ 0.07227698  0.12858619  0.04887397  0.06889654]
 [ 0.10306814  0.25350294 -0.03403696  0.12115229]
 [ 0.15145429  0.08904328 -0.02613462  0.12645757]
 [ 0.756      -1.          0.          0.        ]]


In [0]:
print(Weight_Matrix_DF.to_dict())

NameError: ignored

In [0]:
Weight_Matrix_DF = pd.DataFrame(Weight_Matrix, columns=['pleasantness', 'attention', 'sensitivity', 'aptitude'])

In [0]:
with open('/content/drive/My Drive/Weight_Matrix2.txt', 'w') as f:
	f.write(str(Weight_Matrix_DF.to_dict()))
f.close()